In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np

In [14]:
df = pd.read_csv('mock_student_data.csv')

The first task is to load the file and generate summary statistics for each field as well as probability distributions or histograms. The summary statistics should include mean, median, mode, standard deviation, as well as the number of missing values for each field.


In [95]:
cols = ['Age', 'GPA', 'Days_missed']

In [110]:
df.describe()

,ID,Age,GPA,Days_missed
count,1000.000000,771.000000,779.000000,808.000000
mean,500.500000,16.996109,2.988447,18.011139
std,288.819436,1.458067,0.818249,9.629371
min,1.000000,15.000000,2.000000,2.000000
25%,250.750000,16.000000,2.000000,9.000000
50%,500.500000,17.000000,3.000000,18.000000
75%,750.250000,18.000000,4.000000,27.000000
max,1000.000000,19.000000,4.000000,34.000000


In [111]:
df.mode()

,ID,First_name,Last_name,State,Gender,Age,GPA,Days_missed,Graduated
0,NaN,Amy,Ross,Texas,Female,15,2,6,Yes
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,NaN


In [112]:
df.isnull().sum()

ID               0
First_name       0
Last_name        0
State          116
Gender         226
Age            229
GPA            221
Days_missed    192
Graduated        0
dtype: int64

In [113]:
df.hist()
plt.savefig('histograms')

You will notice that a lot of students are missing gender values . Your task is to infer the gender of the student based on their name. Please use the API at www.genderize.io to infer the gender of each student and generate a new data file.

In [33]:
def genderize(name):
    args = {'name[]': name}
    r = requests.get('https://api.genderize.io/', params=args)
    print(r)
    print(r.json())
    return r.json()

In [16]:
names = list(df['First_name'].unique())

In [38]:
x = 0
name_dict = {}

while len(name_dict) < len(names):
    to_try = []
    for i in range(x,x+10):
        if i <= len(names):
            to_try.append(names[i])
    response = genderize(to_try)
    for r in response:
        name_dict[r['name']] = r['gender']
    x += 10


<Response [200]>
[{'probability': '1.00', 'gender': 'male', 'name': 'Wayne', 'count': 1014}, {'probability': '0.99', 'gender': 'female', 'name': 'Ann', 'count': 1818}, {'probability': '0.99', 'gender': 'male', 'name': 'George', 'count': 2098}, {'probability': '0.98', 'gender': 'male', 'name': 'Jerry', 'count': 1031}, {'probability': '0.79', 'gender': 'female', 'name': 'Andrea', 'count': 5812}, {'probability': '1.00', 'gender': 'female', 'name': 'Annie', 'count': 1120}, {'probability': '1.00', 'gender': 'female', 'name': 'Stephanie', 'count': 4941}, {'probability': '1.00', 'gender': 'female', 'name': 'Janet', 'count': 1579}, {'probability': '1.00', 'gender': 'female', 'name': 'Betty', 'count': 840}, {'probability': '1.00', 'gender': 'male', 'name': 'Henry', 'count': 1067}]
<Response [200]>
[{'probability': '1.00', 'gender': 'male', 'name': 'Samuel', 'count': 1411}, {'probability': '1.00', 'gender': 'male', 'name': 'Todd', 'count': 847}, {'probability': '1.00', 'gender': 'male', 'name': 

In [51]:
name_dict

{'Aaron': 'male',
 'Adam': 'male',
 'Alan': 'male',
 'Albert': 'male',
 'Alice': 'female',
 'Amanda': 'female',
 'Amy': 'female',
 'Andrea': 'female',
 'Andrew': 'male',
 'Angela': 'female',
 'Ann': 'female',
 'Anna': 'female',
 'Anne': 'female',
 'Annie': 'female',
 'Anthony': 'male',
 'Antonio': 'male',
 'Arthur': 'male',
 'Ashley': 'female',
 'Barbara': 'female',
 'Benjamin': 'male',
 'Betty': 'female',
 'Beverly': 'female',
 'Billy': 'male',
 'Bobby': 'male',
 'Bonnie': 'female',
 'Brandon': 'male',
 'Brenda': 'female',
 'Brian': 'male',
 'Bruce': 'male',
 'Carl': 'male',
 'Carlos': 'male',
 'Carol': 'female',
 'Carolyn': 'female',
 'Catherine': 'female',
 'Charles': 'male',
 'Cheryl': 'female',
 'Chris': 'male',
 'Christina': 'female',
 'Christine': 'female',
 'Christopher': 'male',
 'Clarence': 'male',
 'Craig': 'male',
 'Cynthia': 'female',
 'Daniel': 'male',
 'David': 'male',
 'Deborah': 'female',
 'Debra': 'female',
 'Denise': 'female',
 'Dennis': 'male',
 'Diana': 'female',
 

In [41]:
def get_gender(x):
    return name_dict[x]

In [44]:
df['Gender'].isnull().sum()

226

In [49]:
# df['inferred_gender'] = genderize(df['First_name'])
df_genderized = df.copy()
df_genderized['Gender'].fillna(df_genderized['First_name'].apply(get_gender), inplace=True)

In [50]:
df_genderized.head(15)

,ID,First_name,Last_name,State,Gender,Age,GPA,Days_missed,Graduated
0,1,Wayne,Boyd,Florida,Male,19,NaN,9,Yes
1,2,Ann,Matthews,Pennsylvania,Female,18,3,NaN,Yes
2,3,George,Matthews,Texas,male,17,NaN,10,Yes
3,4,Jerry,Ramos,California,Male,15,2,28,No
4,5,Andrea,Carroll,North Carolina,Female,NaN,2,29,No
5,6,Annie,Fisher,Virginia,Female,19,NaN,5,Yes
6,7,Stephanie,Barnes,Oklahoma,Female,NaN,3,20,No
7,8,Janet,Jenkins,Louisiana,Female,15,4,20,Yes
8,9,Betty,Miller,NaN,female,17,3,23,No
9,10,Henry,Woods,Colorado,Male,19,4,14,Yes


In [107]:
df_genderized.to_csv('genderized_data')

NameError: name 'df_genderized' is not defined

You will also notice that some of the other attributes are missing. Your task is to fill in the missing values for Age, GPA, and Days_missed using the following approaches:
Fill in missing values with the mean of the values for that attribute
Fill in missing values with a class-conditional mean (where the class is whether they graduated or not).
Is there a better, more appropriate method for filling in the missing values? If yes, describe and implement it. 

In [83]:
# Approach A
df_a = df.copy()

for col in cols:
    df_a[col].fillna(df_a[col].mean(), inplace=True)

df_a.to_csv('approach_a')

In [91]:
# Approach B
df_b = df.copy()

for col in cols:
    df_b[col].fillna(df_b.groupby("Graduated")[col].transform("mean"), inplace=True)

df_b.to_csv('approach_b')